In [1]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece] nltk 'gem-metrics @ git+https://github.com/GEM-benchmark/GEM-metrics.git' bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/GEM-benchmark/GEM-metrics.git to /tmp/pip-install-r80l3dzf/gem-metrics_2d00e92844db480a934ec168b1040d4f
  Running command git clone -q https://github.com/GEM-benchmark/GEM-metrics.git /tmp/pip-install-r80l3dzf/gem-metrics_2d00e92844db480a934ec168b1040d4f
     |████████████████████████████████| 451 kB 27.1 MB/s 
     |████████████████████████████████| 5.8 MB 78.4 MB/s 
     |████████████████████████████████| 118 kB 65.3 MB/s 
     |████████████████████████████████| 1.3 MB 77.6 MB/s 
     |████████████████████████████████| 60 kB 5.3 MB/s 
     |████████████████████████████████| 44 kB 1.1 MB/s 
     |████████████████████████████████| 10.1 MB 64.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 639 kB 87.7 MB/s 
     |████████████████████████

In [2]:
import numpy as np
import os
from tqdm.auto import tqdm
from typing import List

import torch
import torch.nn as nn
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Text2TextGenerationPipeline, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import MarianTokenizer, MarianModel, MarianMTModel
import datasets 
from datasets import Dataset, load_metric
from transformers.pipelines.pt_utils import KeyDataset
#import gem_metrics

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')


device: cuda


In [3]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/MT_final_project')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [4]:
print(transformers.__version__)

4.25.1


# Load train and dev 

In [5]:
from datasets import load_dataset, load_metric

#mixed_path = os.path.join(os.getcwd() + '/data/TrainMixSmall') #TODO: possibly save dict differently in datascramble
#Mixed_Data_Dict = datasets.load_from_disk(mixed_path)

### Load Train data
train_path = os.path.join(os.getcwd() + '/data/TrainMixDataLargeCONV')
#TrainMixLarge = datasets.load_from_disk(train_path)

### Load TED Dataset 
TED = datasets.load_from_disk(os.path.join(os.getcwd() + '/data/TED_prepended'))
metric = load_metric("sacrebleu")

<ipython-input-5-e92adcda5a7f>:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
# TED Train data (sanity check)
#TEDTrain = Mixed_Data_Dict.filter(lambda example: example["origin"].startswith("Ted"))
#len(TEDTrain)

In [6]:
TEDTrain = TED['train']
TEDDev = TED['dev']
TEDTest = TED['test']

In [7]:
#TrainMixLarge
TEDTrain

Dataset({
    features: ['da', 'en', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 64901
})

In [8]:
TEDDev

Dataset({
    features: ['da', 'en', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3606
})

In [9]:
TEDTest

Dataset({
    features: ['da', 'en', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3606
})

In [ ]:
#metric

# Load model and Tokenizer

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


CONV_TOKEN = 58930

def loadModelAndTokenizer(checkpoint="Helsinki-NLP/opus-mt-da-en", add_special_token=False):
  if checkpoint == "Helsinki-NLP/opus-mt-da-en":
    # load model and tokenizer
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  
  # loading a tuned model
  else:
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

  if add_special_token:
    # add special token
    special_token_dict = {'additional_special_tokens' : ['[CONV]']}
    tokenizer.add_special_tokens(special_token_dict)
    model.resize_token_embeddings(len(tokenizer))

  return model, tokenizer




In [ ]:
model, tokenizer = loadModelAndTokenizer(checkpoint='Helsinki-NLP/opus-mt-da-en', add_special_token=True)

# sanity check
assert len(tokenizer) == 58931

In [ ]:
assert TEDDev['input_ids'][123][0] == CONV_TOKEN
print(f"[CONV] token: {CONV_TOKEN}")
tokenizer.encode('[CONV] hello my name is.')

NameError: ignored

In [11]:
def freezeEncoder(model):
  unfrozen = ['model.decoder', 'lm_head']
  for name, param in model.named_parameters():
    param.requires_grad = False
    for layer in unfrozen:
      if layer in name:
        param.requires_grad = True

  n = model.num_parameters(only_trainable=True) 
  print(f'num trainable parameters: {n}')

In [13]:
def freezeAll(model):
  unfrozen = ['decoder.layers.5.final_layer_norm', 'decoder.layers.5.fc1', 'lm_head']
  for name, param in model.named_parameters():
    param.requires_grad = False
    for layer in unfrozen:
      if layer in name:
        param.requires_grad = True

  n = model.num_parameters(only_trainable=True) 
  print(f'num trainable parameters: {n}')


In [ ]:
# freeze every layer but the final ones
#freezeAll(model)
model.num_parameters(only_trainable=True)

74311168

# Define helper functions and training arguments

### **Curriculum Training Steps**
1. Load in Epoch3 Model (best model)
2. Load in TrainMixLDataLargeCONVCirric2 dataset and train for 2 epochs with this dataset (this dataset contains 50% of the out of domain data as the original)
3. Load in the new model from step 2
4. Load in TrainMixLDataLargeCONVCirric3  and and train for 2 epochs with this dataset (this dataset contains 25% of the out of domain data as the original), save resulting model
5. Load in TedTrainConv dataset and train on 10 epochs (2 * 5, to emulate upsampling), save final model


In [ ]:
#ONLY RUN THIS CELL IF ON STEP 1 and 2 of Ciriculum training
#loads in best model
model_c25_name = 'trained_models/MT_da_en_CL25_1epoch'
model, tokenizer = loadModelAndTokenizer(checkpoint = model_c25_name, add_special_token=False)

In [ ]:

#loads in dataset for step 2 (50% dataset)
mixed_curric50_path = os.path.join(os.getcwd() + '/data/TrainMixLDataLargeCONVCirric2') 
MixedDataCurric50 = datasets.load_from_disk(mixed_curric50_path)


In [ ]:
MixedDataCurric50

Dataset({
    features: ['da', 'en', 'origin', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1304320
})

In [ ]:
# make sure [CONV] token is present
if MixedDataCurric50['origin'][0] == 'Ted_prepended':
  assert MixedDataCurric50['input_ids'][0][0] == CONV_TOKEN

In [ ]:
#ONLY RUN THIS CELL IF ON STEP 3/4 of Ciriculum training

#loads in 50% finetuned model (result from step 2)
#model_C50_name = 'MT_da_en_LargeMixTrainedCirrc50Percent'
#model_C50, tokenizer_C50 = loadModelAndTokenizer(checkpoint = model_C50_name, add_special_token=False)


#loads in dataset for step 4 (25% dataset)
mixed_curric25_path = os.path.join(os.getcwd() + '/data/TrainMixLDataLargeCONVCirric3')
MixedDataCurric25 = datasets.load_from_disk(mixed_curric25_path)


#DON'T FORGET TO SET NUM EPOCHS TO 2

In [ ]:
MixedDataCurric25

# make sure [CONV] token is present
if MixedDataCurric25['origin'][0] == 'Ted_prepended':
  assert MixedDataCurric25['input_ids'][0][0] == CONV_TOKEN

In [ ]:
#ONLY RUN THIS CELL IF ON STEP 5 of Ciriculum training
#model_3e_name = 'MT_da_en_LargeMixTrainedCirrc25Percent'
#model_3e, tokenizer_3e = loadModelAndTokenizer(checkpoint = model_3e_name, add_special_token=False)

#make sure tedtrain is loaded
TEDTrain
#DON'T FORGET TO SET NUM EPOCHS TO whatever works

Dataset({
    features: ['da', 'en', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 64901
})

In [14]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

# Function for computing BLEU score
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
batch_size = 32
model_name = "MT_da_en_CL1_1epoch"
args = Seq2SeqTrainingArguments(
    f"{model_name}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=0,
    save_strategy = 'no',
    num_train_epochs=1,
    predict_with_generate=True,
    fp16 = True,
    push_to_hub = True)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=TEDTrain,
    eval_dataset=TEDDev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.8/dist-packages/huggingface_hub/repository.py:725: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  warnings.warn(
Cloning https://huggingface.co/MichaelHarborg/MT_da_en_CL1_1epoch into local empty directory.
Using cuda_amp half precision backend


In [ ]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
model.num_parameters(only_trainable=True)

74311168

### **Checklist**
* fp16 = True
* [CONV] token added to TEDDev, TEDTrain, tokenizer and model embeddings.
* push_to_hub = True
* model_name = 'Informative_model_name'
* Check if parameters are properly unfrozen/frozen


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: en, da. If en, da are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 64901
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2029
  Number of trainable parameters = 74311168


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.085100,0.139737,47.835700,36.018600


The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: en, da. If en, da are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3606
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2029, training_loss=0.08610645052598227, metrics={'train_runtime': 1111.1808, 'train_samples_per_second': 58.407, 'train_steps_per_second': 1.826, 'total_flos': 3093803131207680.0, 'train_loss': 0.08610645052598227, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub(model_name)

Saving model checkpoint to MT_da_en_CL1_1epoch
Configuration saved in MT_da_en_CL1_1epoch/config.json
Model weights saved in MT_da_en_CL1_1epoch/pytorch_model.bin
tokenizer config file saved in MT_da_en_CL1_1epoch/tokenizer_config.json
Special tokens file saved in MT_da_en_CL1_1epoch/special_tokens_map.json
added tokens file saved in MT_da_en_CL1_1epoch/added_tokens.json
Adding files tracked by Git LFS: ['source.spm', 'target.spm']. This may take a bit of time if the files are large.


KeyboardInterrupt: ignored

In [ ]:
# Use this if push_to_hub = False
def save_model_weights(model, path=os.path.join(os.getcwd(), model_name)):
  model_path = os.path.join(path, 'MODEL_WEIGHTS.pt')
  torch.save(model.state_dict(), model_path)

In [ ]:
save_model_weights(model_3e)

# Load model from drive

In [ ]:
#tuned_model_name = 'MT_da_en_TrainMixLargeCONV_2Epoch'
tuned_model_name = 'MT_da_en_LargeMixTrained4Epochs'

#tuned_model =  AutoModelForSeq2SeqLM.from_pretrained(tuned_model_name)
#tuned_tokenizer = AutoTokenizer.from_pretrained(tuned_model_name)

model_3e_name = 'MT_da_en_LargeMixTrained3Epochs'
model_4e_name = 'MT_da_en_LargeMixTrained4Epochs'


model_3e, tokenizer_3e = loadModelAndTokenizer(checkpoint = model_3e_name, add_special_token=False)
model_base, tokenizer_base = loadModelAndTokenizer() # default arguments are baseline model

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/820k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/788k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

In [ ]:
#model_path = os.path.join(os.getcwd(), 'model_name')

In [ ]:
#model = MarianMTModel.from_pretrained(model_path)
#tokenizer = MarianTokenizer.from_pretrained(model_path)  # Add specific options if needed
# Example for a GPT model
#model = OpenAIGPTDoubleHeadsModel.from_pretrained(output_dir)
#tokenizer = OpenAIGPTTokenizer.from_pretrained(output_dir)

# Validate

In [ ]:
# load TEDDev without the [CONV] token and stripped from '\n'
TEDDev_path = os.path.join(os.getcwd(), 'data/TEDDevTokenized')
TEDDev_vanilla = datasets.load_from_disk(TEDDev_path)

In [ ]:
TEDDev_vanilla

Dataset({
    features: ['da', 'en', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3606
})

In [ ]:
# load 
model_c50, tokenizer_c50 = loadModelAndTokenizer('MT_da_en_3e_CL50_1epoch', add_special_token=False)

loading configuration file MT_da_en_3e_CL50_1epoch/config.json
Model config MarianConfig {
  "_name_or_path": "MT_da_en_3e_CL50_1epoch",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      58929
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 58929,
  "decoder_vocab_size": 58931,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
  

In [ ]:
pipe = Text2TextGenerationPipeline(model=model,  
              tokenizer=tokenizer, 
              device=0, 
              batch_size=16, 
              repetition_penalty=3., 
              num_beams=2,
              length_penalty=1.0, 
              early_stopping=True)

In [15]:
import gem_metrics

def generate(pipeline, data):
  preds = []
  refs = [e.strip('[CONV]') for e in data['en']]
  # get predictions
  for sent in tqdm(pipeline(KeyDataset(data, 'da'), batch_size = 16)):
    preds.append(sent)
  
  return preds, refs

def validate(preds , refs):
  # check if preds is a list of strings
  predictions = []
  if not type(preds[0]) == str:
    for p in preds:
        predictions.append(p[0]['generated_text'].strip('[CONV]'))

  # convert
  if len(predictions) == 0:
    predictions = gem_metrics.texts.Predictions(preds)
  else:
    predictions = gem_metrics.texts.Predictions(predictions)

  refs = gem_metrics.texts.References(refs)
  # compute BLEU score
  result = gem_metrics.compute(predictions, refs, metrics_list=['bleu'])
  return result

In [ ]:
preds, refs = generate(pipe, TEDDev)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


  0%|          | 0/226 [00:00<?, ?it/s]

In [16]:
# remove leading spaces
def postprocess_refs(refs):
  refs2 = [r.strip(' ') for r in refs]
  return refs2

In [17]:
def postprocess_preds(preds):
  # remove _
  predictions = [p[0]['generated_text'].replace('▁',' ') for p in preds]
  # remove special token
  predictions = [p.strip('[CONV]') for p in predictions]
  # remove leading spaces
  predictions = [p.strip(' ') for p in predictions]
  return predictions



In [ ]:
refs2 = postprocess_refs(refs)
preds2 = postprocess_preds(preds)

In [ ]:
refs2[:5]

["To conflicts, it would be U.N.'s peacekeeping forces, and so on.",
 'This means you have to build a water-cooled reactor as a pressure vessel, with steel walls over 20 centimeters thick.',
 '"They\'re not sure this rises to the level of harassment, and they\'re not set up technically to find the device."',
 'Thirdly, make it valuable.',
 'And my parents were very patient, but I had been doing intermittent sojourns to India on a mystical quest.']

In [ ]:
preds[:5]

[[{'generated_text': 'On conflict, it would be the UN▁peace▁keepers and so forth.'}],
 [{'generated_text': 'That means you have to build your water-cooled reactor like a▁pressure cooker with steel walls over an inch thick.'}],
 [{'generated_text': '"They\'re not sure that\'s enough for it to be a violation, and they don\'t have the technical knowledge to find the device."'}],
 [{'generated_text': 'Third, makes it valuable.'}],
 [{'generated_text': 'And my parents were very patient, but I had made several▁temporary trips to▁India on a mystical travel.'}]]

In [ ]:
# compute and show BLEU score
result_TEDdev = validate(preds2, refs2)
print(result_TEDdev)

[I 221206 22:31:28 texts:55] Loading predictions for None
[I 221206 22:31:28 texts:55] Loading references for None
[I 221206 22:31:28 __init__:174] Computing BLEU for None...


{'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 49.09589}


In [18]:
def generateBaseline(pipeline, data):
  preds = []
  refs = [e.strip('[CONV]') for e in data['en']]
  # get predictions
  for sent in tqdm(pipeline(KeyDataset(data, 'da'), batch_size = 16)):
    preds.append(sent)
  
  return preds, refs

In [19]:
def genAndValModels(models : List, tokenizers : List, test):
  results = []
  for i in range(len(models)):
    # create pipeline
    pipe = Text2TextGenerationPipeline(model=models[i],  
              tokenizer=tokenizers[i], 
              device=0, 
              batch_size=16, 
              repetition_penalty=3., 
              num_beams=2,
              length_penalty=1.0, 
              early_stopping=True)
    # get predictions
    preds, refs = generate(pipe, test)
    # postprocess
    refs2 = postprocess_refs(refs)
    preds2 = postprocess_preds(preds)

    # get BLEU
    r = validate(preds2, refs2)
    print("RESULT:\n", r)
    results.append(r)

  return results



In [20]:
baseline_path = 'Helsinki-NLP/opus-mt-da-en'
#layer5_unfrozen_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_MixedSmall_frozen_except_layer_5s')
encoder_frozen_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_EncoderFrozen/MT_da_en_EncoderFrozen.pt')
mixsmall_unfrozen_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_MixedSmall_unfrozen')
output_layer_unfrozen_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_LastLayersUnfrozen/model_weights.pt')

init_prepend_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_prepended/MT_da_en_prepended.pt')

TML_2e_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_TrainMixLargeCONV_2Epoch')
TML_3e_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_LargeMixTrained3Epochs')
TML_4e_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_LargeMixTrained4Epochs')

CL50_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_3e_CL50_1epoch')
CL25_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_CL25_1epoch')
CL1_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_CL1_1epoch')

Ensemble_2e_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_ensemble2E')
Ensemble_4e_path = os.path.join(os.getcwd(), 'trained_models/MT_da_en_ensemble4E')

model_paths = [baseline_path,  
               encoder_frozen_path, 
               mixsmall_unfrozen_path,
               output_layer_unfrozen_path,
               init_prepend_path,
               TML_2e_path,
               TML_3e_path,
               TML_4e_path,
               CL50_path,
               CL25_path,
               CL1_path,
               Ensemble_2e_path,
               Ensemble_4e_path
               ]


In [ ]:
models_for_eval = []
tokenizers_for_eval = []
for path in model_paths:
  if not '.pt' in path:
    model_i, tokenizer_i = loadModelAndTokenizer(path, add_special_token=False)
    models_for_eval.append(model_i)
    tokenizers_for_eval.append(tokenizer_i)
  else:
    
    if 'prepended' not in path:
      model_i, tokenizer_i = loadModelAndTokenizer(baseline_path, add_special_token=False)
      model_i.load_state_dict(torch.load(path))
      models_for_eval.append(model_i)
      tokenizers_for_eval.append(tokenizer_i)
    else:
      model_i, tokenizer_i = loadModelAndTokenizer(baseline_path, add_special_token=True)
      model_i.load_state_dict(torch.load(path))
      models_for_eval.append(model_i)
      tokenizers_for_eval.append(tokenizer_i)


# baseline, CL50, CL1, 3epcochF

In [ ]:
combined_res = genAndValModels(models_for_eval, tokenizers_for_eval, TEDDev)

  0%|          | 0/226 [00:00<?, ?it/s]

[I 221206 16:27:23 texts:55] Loading predictions for None
[I 221206 16:27:23 texts:55] Loading references for None
[I 221206 16:27:23 __init__:174] Computing BLEU for None...


RESULT:
 {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 46.73731}


  0%|          | 0/226 [00:00<?, ?it/s]

[I 221206 16:30:56 texts:55] Loading predictions for None
[I 221206 16:30:56 texts:55] Loading references for None
[I 221206 16:30:56 __init__:174] Computing BLEU for None...


...

[I 221206 16:55:15 texts:55] Loading predictions for None
[I 221206 16:55:15 texts:55] Loading references for None
[I 221206 16:55:15 __init__:174] Computing BLEU for None...


RESULT:
 {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 49.15601}


  0%|          | 0/226 [00:00<?, ?it/s]

[I 221206 16:58:41 texts:55] Loading predictions for None
[I 221206 16:58:41 texts:55] Loading references for None
[I 221206 16:58:41 __init__:174] Computing BLEU for None...


RESULT:
 {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 49.07923}


In [ ]:
print(combined_res)

[{'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 46.73731}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 44.02426}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 42.39199}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 24.69122}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 45.21755}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 48.5613}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 48.84053}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 48.82198}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 49.15601}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 49.07923}]


# Assess catastrophic forgetting

In [ ]:
### Load dataset
CF_path = os.path.join(os.getcwd(),'data/CataForgTestDataTokenized')
CF_test_data = datasets.load_from_disk(CF_path)

In [ ]:
CF_test_data

Dataset({
    features: ['da', 'en', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 6011
})

In [ ]:
### Load models
model_paths = [baseline_path,  
               init_prepend_path,
               CL50_path]

models_for_eval = []
tokenizers_for_eval = []
for path in model_paths:
  if not '.pt' in path:
    model_i, tokenizer_i = loadModelAndTokenizer(path, add_special_token=False)
    models_for_eval.append(model_i)
    tokenizers_for_eval.append(tokenizer_i)
  else:
    
    if 'prepended' not in path:
      model_i, tokenizer_i = loadModelAndTokenizer(baseline_path, add_special_token=False)
      model_i.load_state_dict(torch.load(path))
      models_for_eval.append(model_i)
      tokenizers_for_eval.append(tokenizer_i)
    else:
      model_i, tokenizer_i = loadModelAndTokenizer(baseline_path, add_special_token=True)
      model_i.load_state_dict(torch.load(path))
      models_for_eval.append(model_i)
      tokenizers_for_eval.append(tokenizer_i)

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/820k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/788k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

In [ ]:
cf_res = genAndValModels(models_for_eval, tokenizers_for_eval, CF_test_data)


  0%|          | 0/376 [00:00<?, ?it/s]

[I 221206 18:39:56 texts:55] Loading predictions for None
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.8/dist-
[nltk_data]     packages/data/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[I 221206 18:39:56 texts:55] Loading references for None
[I 221206 18:39:56 __init__:174] Computing BLEU for None...


RESULT:
 {'predictions_file': None, 'N': 6011, 'references_file': None, 'bleu': 42.31221}


  0%|          | 0/376 [00:00<?, ?it/s]

[I 221206 18:46:41 texts:55] Loading predictions for None
[I 221206 18:46:41 texts:55] Loading references for None
[I 221206 18:46:41 __init__:174] Computing BLEU for None...


RESULT:
 {'predictions_file': None, 'N': 6011, 'references_file': None, 'bleu': 38.74475}


  0%|          | 0/376 [00:00<?, ?it/s]

[I 221206 18:53:16 texts:55] Loading predictions for None
[I 221206 18:53:16 texts:55] Loading references for None
[I 221206 18:53:16 __init__:174] Computing BLEU for None...


RESULT:
 {'predictions_file': None, 'N': 6011, 'references_file': None, 'bleu': 44.15624}


In [ ]:
%env CUDA_LAUNCH_BLOCKING=0

env: CUDA_LAUNCH_BLOCKING=0


In [ ]:
! echo $CUDA_LAUNCH_BLOCKING

0


# Run inference on test set of TED2020

In [21]:
model_paths = [baseline_path,  
               init_prepend_path,
               TML_3e_path,
               CL50_path,
               Ensemble_2e_path]

models_for_eval = []
tokenizers_for_eval = []
for path in model_paths:
  if not '.pt' in path:
    model_i, tokenizer_i = loadModelAndTokenizer(path, add_special_token=False)
    models_for_eval.append(model_i)
    tokenizers_for_eval.append(tokenizer_i)
  else:
    
    if 'prepended' not in path:
      model_i, tokenizer_i = loadModelAndTokenizer(baseline_path, add_special_token=False)
      model_i.load_state_dict(torch.load(path))
      models_for_eval.append(model_i)
      tokenizers_for_eval.append(tokenizer_i)
    else:
      model_i, tokenizer_i = loadModelAndTokenizer(baseline_path, add_special_token=True)
      model_i.load_state_dict(torch.load(path))
      models_for_eval.append(model_i)
      tokenizers_for_eval.append(tokenizer_i)

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/820k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/788k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

In [22]:
tedtest_res = genAndValModels(models_for_eval, tokenizers_for_eval, TEDTest)

  0%|          | 0/226 [00:00<?, ?it/s]

[I 221207 03:06:47 texts:55] Loading predictions for None
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.8/dist-
[nltk_data]     packages/data/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[I 221207 03:06:48 texts:55] Loading references for None
[I 221207 03:06:48 __init__:174] Computing BLEU for None...


RESULT:
 {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 46.62962}


  0%|          | 0/226 [00:00<?, ?it/s]

[I 221207 03:10:22 texts:55] Loading predictions for None
[I 221207 03:10:22 texts:55] Loading references for None
[I 221207 03:10:22 __init__:174] Computing BLEU for None...


...

[I 221207 03:17:23 texts:55] Loading predictions for None
[I 221207 03:17:23 texts:55] Loading references for None
[I 221207 03:17:23 __init__:174] Computing BLEU for None...


RESULT:
 {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 48.8008}


  0%|          | 0/226 [00:00<?, ?it/s]

[I 221207 03:20:52 texts:55] Loading predictions for None
[I 221207 03:20:52 texts:55] Loading references for None
[I 221207 03:20:52 __init__:174] Computing BLEU for None...


RESULT:
 {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 48.15786}


In [23]:
print(tedtest_res)

[{'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 46.62962}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 45.34992}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 48.65707}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 48.8008}, {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 48.15786}]


In [25]:
model = AutoModelForSeq2SeqLM.from_pretrained(Ensemble_4e_path)
tokenizer = AutoTokenizer.from_pretrained(Ensemble_4e_path)

In [26]:
ens_res = genAndValModels([model], [tokenizer], TEDTest)

  0%|          | 0/226 [00:00<?, ?it/s]

[I 221207 04:05:21 texts:55] Loading predictions for None
[I 221207 04:05:21 texts:55] Loading references for None
[I 221207 04:05:21 __init__:174] Computing BLEU for None...


RESULT:
 {'predictions_file': None, 'N': 3606, 'references_file': None, 'bleu': 48.20503}
